In [1]:
import pandas as pd
import plotly.express as px
# Biblioteca para plotar mapa com gráfico de calor
import folium
from folium.plugins import HeatMap
from folium import plugins


In [2]:
# carregar bancos de dados
df = pd.read_csv(r"D:\Projetos_Analytics\01.cooperativas\01.Coop_Brasil\base.csv")

In [3]:
dados = df
#transformar a coluna database, de string para datatime
dados['data_base'] = pd.to_datetime(dados['data_base'], format='%Y-%m-%d')
# Deletar colunas desnecessárias para análise
dados = dados.drop(columns=[ 'ocupacao', 'cnae_secao', 'cnae_subclasse', 'porte',
'a_vencer_ate_90_dias', 'a_vencer_de_91_ate_360_dias', 'a_vencer_de_361_ate_1080_dias', 
'a_vencer_de_1081_ate_1800_dias','a_vencer_acima_de_5400_dias', 'vencido_acima_de_15_dias',
])
dados = dados.drop(columns=[ 'origem', 'indexador', 'numero_de_operacoes', 'a_vencer_de_1801_ate_5400_dias'])
# transformar as colunas numéricas, que estão no formato string para float.
## Os números estão separados por virgulas, por isto foi usado repleace para transformar virgula em ponto 
dados['carteira_ativa'] = dados['carteira_ativa'].str.replace(',', '.').astype(float)
dados['carteira_inadimplida_arrastada'] = dados['carteira_inadimplida_arrastada'].str.replace(',', '.').astype(float)
dados['ativo_problematico'] = dados['ativo_problematico'].str.replace(',', '.').astype(float)


Preparar os dados para analisar a carteira geral por estado por tipo de instituição e postetiormente comparar com o cooperativismo.
será criado uma visão do total por estado e sua evolução e uma visão da share do cooperativimo por estado e sua evolução.


# envidamento total por estados:


In [162]:
# Total do envidamento por estados:
## Agrupar
dados_group = dados.groupby(['data_base', 'uf'])[['carteira_ativa', 'carteira_inadimplida_arrastada']].sum().reset_index()

In [29]:
# período único - para posterior tabela, de filtros:
periodo  = dados_group['data_base'].unique()
periodo

<DatetimeArray>
['2022-06-30 00:00:00', '2022-07-31 00:00:00', '2022-08-31 00:00:00',
 '2022-09-30 00:00:00', '2022-10-31 00:00:00', '2022-11-30 00:00:00',
 '2022-12-31 00:00:00', '2023-01-31 00:00:00', '2023-02-28 00:00:00',
 '2023-03-31 00:00:00', '2023-04-30 00:00:00', '2023-05-31 00:00:00']
Length: 12, dtype: datetime64[ns]

In [179]:
dados_group = dados_group.loc[dados_group['data_base']== '2023-05-31 00:00:00']

In [166]:
#coordenada geodesidas da localização dos estados Brasileiros
# fonte: https://gist.github.com/ricardobeat/674646
local = [
     ['AC', -8.77, -70.55]
  ,  ['AL', -9.71, -35.73]
  ,  ['AM', -3.07, -61.66]
  ,  ['AP',  1.41, -51.77]
  ,  ['BA',-12.96, -38.51]
  ,  ['CE', -3.71, -38.54]
  ,  ['DF',-15.83, -47.86]
  ,  ['ES',-19.19, -40.34]
  ,  ['GO',-16.64, -49.31]
  ,  ['MA', -2.55, -44.30]
  ,  ['MT',-12.64, -55.42]
  ,  ['MS',-20.51, -54.54]
  ,  ['MG',-18.10, -44.38]
  ,  ['PA', -5.53, -52.29]
  ,  ['PB', -7.06, -35.55]
  ,  ['PR',-24.89, -51.55]
  ,  ['PE',-8.28, -35.07]
  ,  ['PI', -8.28, -43.68]
  ,  ['RJ',-22.84, -43.15]
  ,  ['RN', -5.22, -36.52]
  ,  ['RO', -11.22, -62.80]
  ,  ['RS', -30.01, -51.22]
  ,  [ 'RR', 1.89, -61.22]
  ,  ['SC', -27.33, -49.44]
  ,  ['SE', -10.90, -37.07]
  ,  ['SP',-23.55, -46.64]
  ,  ['TO',-10.25, -48.25]
]
local = pd.DataFrame(local, columns =[ 'uf', 'latitude', 'logitude'])
#merge da localização com o df 'dados_group'
dados_group = dados_group.merge(local, on = 'uf', how = 'left')

In [190]:
#criar mapa base do Brasil:
basemap = folium.Map(
    width = '100%',
    heigth = '100%',
    location = [-15.788494, -47.899873],
    zoom_start = 4
)
#coordenadas para o mapa:
coordenadas = dados_group[['latitude', 'logitude', 'carteira_ativa']].values.tolist()
#plotando os dados de coordenadas no mapa base do Brasil
HeatMap(coordenadas, radius = 30).add_to(basemap)
basemap


In [4]:
# teste mapa shapefile 
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [180]:
# plotar gráfico com o valor da carteira por estado por estados:
fig = px.bar(dados_group, x = 'uf', y = 'carteira_ativa', title = 'Saldo por UF')
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'uf=%{x}<br>carteira_ativa=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS',
                          'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC',
                          'SE', 'SP', 'TO'], dtype=object),
              'xaxis': 'x',
              'y': array([1.36590911e+10, 4.08496844e+10, 5.03244919e+10, 1.29380921e+10,
                          2.02739255e+11, 1.16673701e+11, 1.40646351e+11, 8.47100340e+10,
                          2.40696789e+11, 8.02942665e+10, 4.93693028e+11, 1.03871233e+11,
                          1.87594552e+11, 1.08962726e+11, 5.43694798e+10, 1.24335469e+11,
                          4.35148212e+10, 4.14204515e+11, 4.69072586e+11, 5.20992590e+10,
                          4.63280015e+10, 1.05275800e+10, 3.80342360e+11, 2.92573803e+11,
                          3.19300591e+10, 1.54561210e+12, 3.67796188e+10]),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Saldo por UF'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'uf'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'carteira_ativa'}}}
})

# Analise a carteira de crédito cooperativo:

In [193]:
# Total do envidamento por estados:
## Agrupar
dados_coop = dados.groupby(['data_base', 'uf', 'tcb'])[['carteira_ativa', 'carteira_inadimplida_arrastada']].sum().reset_index()
dados_coop = dados_coop.loc[dados_coop['data_base']== '2023-05-31 00:00:00']
dados_coop = dados_coop.loc[dados_coop['tcb']== 'Cooperativas']
dados_coop = dados_coop.merge(local, on = 'uf', how = 'left')

In [199]:
#criar mapa base do Brasil:
basemap2 = folium.Map(
    width = '100%',
    heigth = '100%',
    location = [-15.788494, -47.899873],
    zoom_start = 4
)
#coordenadas para o mapa:
coordenadas2 = dados_coop[['latitude', 'logitude', 'carteira_ativa']].values.tolist()
#plotando os dados de coordenadas no mapa base do Brasil
HeatMap(coordenadas2, radius = 20).add_to(basemap2)
basemap2

In [195]:
# plotar gráfico com o valor da carteira por estado por estados:
fig = px.bar(dados_coop, x = 'uf', y = 'carteira_ativa', title = 'Saldo por UF')
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'uf=%{x}<br>carteira_ativa=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS',
                          'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC',
                          'SE', 'SP', 'TO'], dtype=object),
              'xaxis': 'x',
              'y': array([8.32280174e+08, 1.05881838e+09, 8.53448723e+08, 1.45756527e+08,
                          3.61622755e+09, 1.04256382e+09, 2.28507675e+09, 1.06640898e+10,
                          1.76199272e+10, 8.37431366e+08, 4.87498704e+10, 9.06423337e+09,
                          2.64811453e+10, 4.35065474e+09, 2.59063277e+09, 1.89011849e+09,
                          2.40776825e+08, 5.24343050e+10, 4.76103843e+09, 1.13979213e+09,
                          8.35707213e+09, 2.01898115e+08, 6.03138773e+10, 5.10261331e+10,
                          3.69456073e+08, 3.87135518e+10, 2.15339993e+09]),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Saldo por UF'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'uf'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'carteira_ativa'}}}
})

# Benchmark do cooperativismo:


In [1]:
# analisar a variação do crescimento das cooperativas:
# crescimento absoluto 
dados_coop2 = dados.groupby(['data_base', 'uf', 'tcb'])[['carteira_ativa', 'carteira_inadimplida_arrastada']].sum().reset_index()
dados_coop2 = dados_coop2.loc[dados_coop2['data_base']== '2022-06-30 00:00:00']
dados_coop2 = dados_coop2.loc[dados_coop2['tcb']== 'Cooperativas']
dados_coop2 = dados_coop2.merge(local, on = 'uf', how = 'left')

NameError: name 'dados' is not defined

In [197]:
dados_bench.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10614138 entries, 0 to 10614137
Data columns (total 10 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   Unnamed: 0                      int64         
 1   data_base                       datetime64[ns]
 2   uf                              object        
 3   tcb                             object        
 4   sr                              object        
 5   cliente                         object        
 6   modalidade                      object        
 7   carteira_ativa                  float64       
 8   carteira_inadimplida_arrastada  float64       
 9   ativo_problematico              float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 809.8+ MB


In [ ]:
da